# Dataframe Editing

### 필요 라이브러리 임포트

In [1]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Kkma, Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

kkma=Kkma()
okt =Okt()

cv = CountVectorizer()
tfidf = TfidfVectorizer()

### 데이터 불러오기 및 첫 정제

In [23]:
filedir = './data/news_dfv02.csv'
#-------------------------------------------------------
#-------------------------------------------------------
# loading crawled csv
def loadcsv(filedir):
    crawldf = pd.read_csv (filedir, sep='w')
    crawldf = crawldf.dropna()
    # there are 5columns in CrawledDF -> 'media', 'date', 'title', 'article_original', 'url'
    firstdf = crawldf[['media','date','title','article_original','url']].copy()
    # reseting index in order after droping NA
    firstdf = firstdf.reset_index(drop=True)
    return firstdf
    
    

In [49]:
firstdf = loadcsv(filedir)
firstdf

,media,date,title,article_original,url,tokenized_sents,common,sim
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[사회자, 소개, 방탄소년단, 모습, 마지막, 날, 미국, 뉴욕, 맨해튼, 타임스스...","[1, 2]","1, 2"
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[음악, 장르, 스스로, 재창, 인상, 영향력, 행사, 음악, 산업, 변화, 아티스...",[0],0
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[경자년, 새해, 팝, 스타, 세계, 무대, 공략, 계속, 전망, 지난해, 전, 세...","[9, 19, 11, 0]","9, 19, 11, 0"
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[팝, 대한, 관심, 전, 세계, 서울시, 팝, 명소, 선정, 홍보, 멀리, 한국,...",[],
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[골든글로브, 시상식, 한국, 최초, 외국어, 영화상, 수상한, 기생충, 봉준호, ...",[],
...,...,...,...,...,...,...,...,...
853,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[코로나, 대유행, 고통, 한해, 방탄소년단, 위로, 케이팝, 시장, 상대, 기획사...","[852, 842, 830]","852, 842, 830"
854,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[세계, 모든, 날, 지도, 오늘, 오늘, 낫, 투데이, 가운데, 방탄소년단, 자체...",[834],834
855,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[한국, 문화, 저력, 바닥, 전례, 세계, 기록, 약속, 일제, 동시, 코로나, ...","[847, 848]","847, 848"
856,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[박영선, 중소, 벤처기업, 부, 장관, 발표, 신년사, 최선, 더, 힘, 내, 송...",[],


### 토크나이징

In [36]:
filedir = './data/news_dfv02.csv'
StopWordsDir = "./data/stopwords.txt"

# loading stopwords from StopWordsDirectory
def stopwords(StopWordsDir):
    with open (StopWordsDir, "r", encoding = 'utf-8') as myfile:
        data=myfile.read()
    stop_words=data.split('\n')
    
    return stop_words
    
# tokenize without stopwords function   
def cleantokenize(df):
    result = []
    
    for w in okt.nouns(df):
        if w not in stopwords(StopWordsDir):
            result.append(w)
    
    return result

# apply cleantokenize function to a dataframe
def applytokenize(df):
    df['tokenized_sents'] = df.apply(lambda row: cleantokenize(row['article_original']), axis=1)
    return df



In [48]:
tokenized_df = applytokenize(firstdf)
tokenized_df

,media,date,title,article_original,url,tokenized_sents,common,sim
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[사회자, 소개, 방탄소년단, 모습, 마지막, 날, 미국, 뉴욕, 맨해튼, 타임스스...","[1, 2]","1, 2"
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[음악, 장르, 스스로, 재창, 인상, 영향력, 행사, 음악, 산업, 변화, 아티스...",[0],0
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[경자년, 새해, 팝, 스타, 세계, 무대, 공략, 계속, 전망, 지난해, 전, 세...","[9, 19, 11, 0]","9, 19, 11, 0"
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[팝, 대한, 관심, 전, 세계, 서울시, 팝, 명소, 선정, 홍보, 멀리, 한국,...",[],
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[골든글로브, 시상식, 한국, 최초, 외국어, 영화상, 수상한, 기생충, 봉준호, ...",[],
...,...,...,...,...,...,...,...,...
853,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[코로나, 대유행, 고통, 한해, 방탄소년단, 위로, 케이팝, 시장, 상대, 기획사...","[852, 842, 830]","852, 842, 830"
854,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[세계, 모든, 날, 지도, 오늘, 오늘, 낫, 투데이, 가운데, 방탄소년단, 자체...",[834],834
855,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[한국, 문화, 저력, 바닥, 전례, 세계, 기록, 약속, 일제, 동시, 코로나, ...","[847, 848]","847, 848"
856,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[박영선, 중소, 벤처기업, 부, 장관, 발표, 신년사, 최선, 더, 힘, 내, 송...",[],


### 문서 유사도

In [45]:
# 코사인 유사도를 통한 유사한 기사 뽑아내는 함수
def getcosine (df):
    article_data =df.astype(str)
    tfidf_matrix = tfidf.fit_transform(article_data)
    # 코사인유사도 공식
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    return cosine_sim

# 각 인덱스에 해당하는 기사와 유사한 기사들의 인덱스 추출
def similarity(index,df):
    result= []
    cosine_sim=getcosine(df)
 
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # 비슷한 유사도에 해당하는 기사 인덱스 9개 추출
    sim_scores = sim_scores[1:10]
    article_indices = [i[0] for i in sim_scores]
    # 날짜가 너무 퍼지지 않게 Index 범위 조정
    for n in article_indices:
        if n < (int(index)+30) and n > (int(index)-30):
            result.append(n)
    return result

# 해당 컬럼에 해당하는 모든 열에 대하여 수행
def similarlist (df):
    new_list = []
    for i in range(len(df)):
        new_list.append(similarity(i,df))
    return new_list

# 유사도 시각화를 위한 컬럼 추가
def applysim (df):
    df['common'] = similarlist(df['article_original'])
    # List 로 된 Similar Article String 타입으로
    df['sim'] = [', '.join(map(str, l)) for l in df['common']]
    return df

In [47]:
applysim_df = applysim (tokenized_df)
applysim_df

,media,date,title,article_original,url,tokenized_sents,common,sim
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[사회자, 소개, 방탄소년단, 모습, 마지막, 날, 미국, 뉴욕, 맨해튼, 타임스스...","[1, 2]","1, 2"
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[음악, 장르, 스스로, 재창, 인상, 영향력, 행사, 음악, 산업, 변화, 아티스...",[0],0
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[경자년, 새해, 팝, 스타, 세계, 무대, 공략, 계속, 전망, 지난해, 전, 세...","[9, 19, 11, 0]","9, 19, 11, 0"
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[팝, 대한, 관심, 전, 세계, 서울시, 팝, 명소, 선정, 홍보, 멀리, 한국,...",[],
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[골든글로브, 시상식, 한국, 최초, 외국어, 영화상, 수상한, 기생충, 봉준호, ...",[],
...,...,...,...,...,...,...,...,...
853,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[코로나, 대유행, 고통, 한해, 방탄소년단, 위로, 케이팝, 시장, 상대, 기획사...","[852, 842, 830]","852, 842, 830"
854,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[세계, 모든, 날, 지도, 오늘, 오늘, 낫, 투데이, 가운데, 방탄소년단, 자체...",[834],834
855,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[한국, 문화, 저력, 바닥, 전례, 세계, 기록, 약속, 일제, 동시, 코로나, ...","[847, 848]","847, 848"
856,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[박영선, 중소, 벤처기업, 부, 장관, 발표, 신년사, 최선, 더, 힘, 내, 송...",[],


### 데이터프레임 최종 정제

In [51]:
# 존재했던 데이터프레임에서 유사도가 겹치지 않는 기사들을 drop, 그리고 재정렬 
# 재정렬하면 유사도 인덱스또한 변동 사항이 때문에 다시 받아야한다
def recleanse(df):
    newdf = df.drop(columns = 'common')
    newdf = newdf.replace('', np.nan)
    newdf = newdf.dropna()
    newdf = newdf.drop(columns = 'sim')
    newdf = newdf.reset_index (drop=True)
    return newdf

In [52]:
cleansed_df = recleanse(applysim_df)

In [53]:
def recosinesim(df):
    df['sim'] = similarlist(df['tokenized_sents'])
    df['sim'] = [', '.join(map(str, l)) for l in df['sim']]
    
    simcount = pd.Series(df['sim'].str.count('\w+'), name = 'simcount')
    finaldf = pd.concat([df, simcount], axis=1)
    return finaldf


In [54]:
final_df = recosinesim(cleansed_df)
final_df

,media,date,title,article_original,url,tokenized_sents,sim,simcount
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[사회자, 소개, 방탄소년단, 모습, 마지막, 날, 미국, 뉴욕, 맨해튼, 타임스스...","1, 2",2
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[음악, 장르, 스스로, 재창, 인상, 영향력, 행사, 음악, 산업, 변화, 아티스...",0,1
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[경자년, 새해, 팝, 스타, 세계, 무대, 공략, 계속, 전망, 지난해, 전, 세...","6, 8, 26",3
3,조선일보,2020-01-07,BTS 상표권 분쟁 일단락 신세계 상표권 포기 방탄소년단 응원,‘‘BTS’ 상표권을 두고 그룹 방탄소년단BTS 소속사 빅히트엔터테인먼트이하 빅히트...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[상표권, 그룹, 방탄소년단, 소속사, 빅히트, 엔터테인먼트, 이하, 빅히트, 분쟁...",4,1
4,한겨레,2020-01-07,신세계 BTS 상표권 모두 포기,방탄소년단BTS 소속사 빅히트엔터테인먼트와 ‘비티에스’BTS 상표권을 두고 분쟁 중...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[방탄소년단, 소속사, 빅히트, 엔터테인먼트, 비, 티에스, 상표권, 분쟁, 중, ...",3,1
...,...,...,...,...,...,...,...,...
746,한겨레,2020-12-26,방탄학BTSology BTS 연구는 이미 시작되었다,방탄소년단BTS의 전세계적인 인기는 이제 낯설지 않다. 2017년 처음 미국 무대에...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[방탄소년단, 전세계, 인기, 이제, 처음, 미국, 무대, 선, 뒤, 지난, 빌보드...","748, 747, 728, 730, 732",5
747,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[코로나, 대유행, 고통, 한해, 방탄소년단, 위로, 케이팝, 시장, 상대, 기획사...","746, 725, 737",3
748,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[세계, 모든, 날, 지도, 오늘, 오늘, 낫, 투데이, 가운데, 방탄소년단, 자체...","746, 747",2
749,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...,"[한국, 문화, 저력, 바닥, 전례, 세계, 기록, 약속, 일제, 동시, 코로나, ...","741, 742, 729",3


### Clustering, Summarization 단계를 위해 저장

In [56]:
# 내 마음속에 저장
savedir = './data/final_df.csv'
def dftocsv (final_df, savedir):
    final_df.to_csv (savedir, index = False, encoding = 'utf-8-sig')
    # 쓸데없는 (유사도가 겹치지않는) 기사들을 제거 한 최종 데이터 프레임
    
dftocsv (final_df, savedir)